In [1]:
# подключим необходимые библиотеки
# Единицы измерения
from pint import UnitRegistry
ureg = UnitRegistry()

# Универсальная газовая постоянная
from scipy.constants import R, pi
print(R)
# Единица измерения для универсальной газовой постоянной
R_UOM = ureg.joule / (ureg.mol * ureg.kelvin)
R_UOM

# Толщина материала (ткани ПВХ, которую используют для производства ВГУ)
thickness_PVC = 0.7 * ureg.millimeter

8.314462618






# Газопроницаемость

Оценка газопроницаемости тканей с двусторонним покрыттием (резиновым или ПВХ) для проекта СТО ВГУ ОТУ.

Будет проведена попытка оценки двумя методами:

- первый - на основании установленного требования по ГОСТ 8973 - 0,002 м² / (Па / с)
- второй - на основании установленного требования к уменьшению давления в ВГУ - 10 % за 6 часов

## Исходные данные

### Газопроницаемость реальных материалов

Газопроницаемость ПВХ-ткани в открытых источниках найти не удалось.

Газопроницаемость ПВХ

 Газ | Значение | Единица измерения | Ссылка
:---:|:--------:|:-----------------:|:-------:
Кислород | 0.198 - 0.790 | мл \* м / (м² \* сутки * Па) | [ссылка](https://www.quora.com/How-long-can-a-human-survive-in-an-inflatable-plastic-bubble-with-a-2-meter-diameter)
Кислород | 3100 | см³ \* микрометр / (м² \* сутки * бар) | [ссылка]( http://exclusive.multibriefs.com/content/novel-medical-plastics-packaging-material-and-process-advances/engineering)


Газопроницаемость других материалов (для сравнения)

 Материал | Газ | Значение | Единица измерения | Ссылка | Примечание
:--------:|:---:|:--------:|:-----------------:|:------:|:-----------:
Поливинилиденхлорид | Кислород | 2.6 | см³ \* 0.2 микрометра / (м² \* сутки * атмосферу) | [ссылка](https://www.quora.com/How-long-can-a-human-survive-in-an-inflatable-plastic-bubble-with-a-2-meter-diameter) | Указано, что хороший барьер
Поливинилиденхлорид | Кислород | 0.001 - 0.03 | мл \* м / (м² \* сутки * Па) | [ссылка](https://www.quora.com/How-long-can-a-human-survive-in-an-inflatable-plastic-bubble-with-a-2-meter-diameter)




In [2]:
# Проверим исходные данные, полученные из разных источников
# Сравнить по самим данным достаточно сложно, т.к. единицы измерения очень отличаются
# для перевода воспользуемся математическим пакетом (автоматически переведем единицы измерения) pint
# Единица измерения первой строки таблицы "Газопроницаемость ПВХ"
p_pvc1_UOM = ureg.milliliter * ureg.meter / (ureg.meter * ureg.meter * ureg.day * ureg.pascal)     # UOM - unit of measure
p_pvc1_UOM

<Unit('milliliter / day / meter / pascal')>

In [3]:
# Единица измерения второй строки таблицы "Газопроницаемость ПВХ"
p_pvc2_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter * ureg.micrometer / (ureg.meter * ureg.meter * ureg.day * ureg.bar)
p_pvc2_UOM

<Unit('centimeter ** 3 * micrometer / bar / day / meter ** 2')>

In [4]:
# Перводим значение второй строки в единицы измерения первой строки
permiability_PVC_2 = (3100 * p_pvc2_UOM).to(p_pvc1_UOM)
permiability_PVC_2

3.1000000000000006e-08 <Unit('milliliter / day / meter / pascal')>

In [5]:
permiability_PVC_1 = 0.198 * p_pvc1_UOM
permiability_PVC_1

0.198 <Unit('milliliter / day / meter / pascal')>

In [6]:
permiability_PVC_1 / permiability_PVC_2

6387096.774193548 <Unit('dimensionless')>

In [7]:
# Единица измерения первой строки таблицы "Газопроницаемость других материалов"
p_1_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter * 0.2 * ureg.micrometer / (ureg.meter * ureg.meter * ureg.day * ureg.atmosphere) # UOM - unit of measure
p_1_UOM

0.2 <Unit('centimeter ** 3 * micrometer / day / meter ** 2 / standard_atmosphere')>

In [8]:
# Единица измерения второй строки таблицы "Газопроницаемость других материалов"
p_2_UOM = ureg.milliliter * ureg.meter / (ureg.meter * ureg.meter * ureg.day * ureg.pascal)
p_2_UOM

<Unit('milliliter / day / meter / pascal')>

In [9]:
# Переводим значение первой строки в единицы измерения второй строки
permiability_PVdC_1 = (2.6 * p_1_UOM).to(p_2_UOM)
permiability_PVdC_1

5.1320009869232674e-12 <Unit('milliliter / day / meter / pascal')>

In [10]:
# Значение второй строки
permiability_PVdC_2 = 0.001 * p_2_UOM
permiability_PVdC_2

0.001 <Unit('milliliter / day / meter / pascal')>

In [11]:
# Сравним значение первой строки со значением второй строки в одинаковых единицах измерения
permiability_PVdC_2 / permiability_PVdC_1

194855769.23076922 <Unit('dimensionless')>

In [12]:
f'Первое значение {permiability_PVdC_1.magnitude:.2e} в {(permiability_PVdC_2 / permiability_PVdC_1).magnitude:.2e} раз меньше второго значения {permiability_PVdC_2.magnitude}'

'Первое значение 5.13e-12 в 1.95e+08 раз меньше второго значения 0.001'

**Вывод**

Ориентироваться на показания, полученные из открытых источников нельзя:

- не известно по какой методике проведен контроль;
- не известна достоверность результатов (данные отличаются в несколько раз).

In [13]:
# Максимум по ГОСТ 23553 2 * 10**(-12) м / (Па * с)
gost23553_max_permiability = 2 * 10**(-12) * ureg.meter / (ureg.pascal * ureg.second)

## Метод 1

В первом методе выведем значения газопроницаемости по двум оставшимся ГОСТам (ГОСТ 23553 и ГОСТ 53656.2) по значению, которое установлено сейчас для первого ГОСТа (ГОСТ 8973)

Значение воздухопроницаемости по ГОСТ 8973 - 0,002 м² / (Па / с)

### ГОСТ 8973

In [14]:
gost8973_UOM = ureg.meter * ureg.meter / (ureg.pascal * ureg.second)
gost8973_permiability = 0.02 * gost8973_UOM

### ГОСТ 23553

Единица измерения - м / (Па * сек.). Для перевода необходимо знать толщину материала. Смысл газопроницаемости по ГОСТ 23553 получается похож на скорость проникновения газов по ГОСТ Р 53656.2 (GPR).

In [15]:
# Единицы измерения
gost23553_P_UOM = ureg.meter / (ureg.pascal * ureg.second)

In [16]:
# Переводим значение по ГОСТ 8973 в значения по ГОСТ 23553 с учетом толщины
gost23553_permiability = (gost8973_permiability / thickness_PVC).to(gost23553_P_UOM)
gost23553_permiability

28.571428571428573 <Unit('meter / pascal / second')>

### ГОСТ Р 53656.2

In [17]:
# Основные единицы измерения по ГОСТ 53656 для
# Р - газопроницаемость (скорость проникновения газа на обратную единицу толщины - чем толще, тем ниже газопроницаемость)
# GTR - скорость проникновения газа (отличается тем, что не зависит от толщины)
gost53656_P_UOM = ureg.mol * ureg.meter / (ureg.meter * ureg.meter * ureg.second * ureg.pascal)
gost53656_GTR_UOM = ureg.mol / (ureg.meter * ureg.meter * ureg.second * ureg.pascal)

gost53656_P_alt_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter * ureg.millimeter / (ureg.meter * ureg.meter * ureg.day * ureg.atmosphere)
gost53656_GTR_alt_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter / (ureg.meter * ureg.meter * ureg.day * ureg.atmosphere)

In [18]:
# Перевод значения по ГОСТ 8973 в значения по ГОСТ 53656 (альтернативные единицы измерения, приведенные в п. А.6.2)
# TODO Подумать, надо ли разделять только на кислород или считать как для всего воздуха
gost53656_permiability_alt = (gost8973_permiability).to(gost53656_P_alt_UOM)
gost53656_permiability_alt

1.7508959999999997e+17 <Unit('centimeter ** 3 * millimeter / day / meter ** 2 / standard_atmosphere')>

In [19]:
# Переведем в основные единицы измерения по ГОСТ 53656
# Чтобы перевести необходимо привести к нормальным условиям
gost53656_permiability = gost53656_permiability_alt * ureg.atmosphere / (R * R_UOM * 293 * ureg.kelvin)
gost53656_permiability.ito(gost53656_P_UOM)
gost53656_permiability

0.8318495819064042 <Unit('mole / meter / pascal / second')>

## Метод 2

На основании установленного требования к уменьшению давления в ВГУ - 10 % за 6 часов

In [20]:
# Исходные данные для метода 2
diameter_VGU = 1.4 * ureg.meter # Рассматриваем самый большой диаметр, т.к. у него самая большая площадь, соответсвенно самые большие потери
length_VGU = 1.5 * ureg.meter

# Площадь поверхности ВГУ
area_VGU = pi * diameter_VGU * length_VGU + 2 * pi * diameter_VGU ** 2 # Периметр на длину плюс два торца

# Объем ВГУ
volume_VGU = pi * diameter_VGU**2 * length_VGU

# Рабочее давление
VGU_pressure = 35 * 10**3 * ureg.pascal

# Изменение давления
pressure_change = VGU_pressure * 0.1 

# Количество газа 
nu = (pressure_change * volume_VGU / (R * R_UOM * 293 * ureg.kelvin )).to(ureg.mol)
print(nu)

# Объем газа в нормальных условиях
volume = (nu * R * R_UOM * 293 * ureg.kelvin / (ureg.atmosphere)).to (ureg.meter * ureg.meter * ureg.meter)
print(f'Объем потерь газа из ВГУ в нормальных условиях {volume}')

# Продолжительность
period = 6 * ureg.hour

13.269771423319801 mole
Объем потерь газа из ВГУ в нормальных условиях 0.319042570001865 meter ** 3


### ГОСТ 8973

In [21]:
gost8973_permiability_m2 = (volume * thickness_PVC / (35 * 10**3 * ureg.pascal * period * area_VGU)).to(gost8973_UOM)
gost8973_permiability_m2

1.5619909583037674e-14 <Unit('meter ** 2 / pascal / second')>

### ГОСТ 23553

In [22]:
gost23553_permiability_m2 = (volume / (35 * 10**3 * ureg.pascal * period * area_VGU)).to(gost23553_P_UOM)
gost23553_permiability_m2

2.2314156547196674e-11 <Unit('meter / pascal / second')>

### ГОСТ 53656.2

In [23]:
gost53656_permiability_m2 = (gost8973_permiability_m2 * ureg.atmosphere / (R * R_UOM * 293 * ureg.kelvin)).to(gost53656_P_UOM)
gost53656_permiability_m2

6.496707628032863e-13 <Unit('mole / meter / pascal / second')>

## Итоги

### Метод 1

In [24]:
import pandas as pd

In [27]:
dates = pd.DataFrame(data=[
        ['ИД: ПВХ 1', permiability_PVC_1.magnitude, permiability_PVC_1.units, '', permiability_PVC_1.to(gost8973_UOM).magnitude],
        ['ИД: ПВХ 2', permiability_PVC_2.magnitude, permiability_PVC_2.units, '', permiability_PVC_2.to(gost8973_UOM).magnitude],
        ['ИД: Максимум по ГОСТ 23553', gost23553_max_permiability.magnitude, gost23553_max_permiability.units, f'Перевод в единицы по ГОСТ 8973 при толщине {thickness_PVC}', (gost23553_max_permiability * thickness_PVC).to(gost8973_UOM).magnitude],
        ['М1 ГОСТ 8973', f'{gost8973_permiability.magnitude}', gost8973_permiability.units, '', gost8973_permiability.magnitude],
        ['М1 ГОСТ 23553', gost23553_permiability.magnitude, gost23553_permiability.units, f'Соответсвует значению по ГОСТ 8973 при толщине {thickness_PVC}. Значение намного !!! больше, чем допускает ГОСТ 23553 (2*10**(-12) м/(Па * сек.)). Единица измерения как у скорости проникновения газов в ГОСТ Р 53656 (GTR)', (gost23553_permiability * thickness_PVC).to(gost8973_UOM).magnitude],
        ['М1 ГОСТ Р 53656', f'{gost53656_permiability.magnitude:.2}', gost53656_permiability.units, 'Соответствует значению по ГОСТ 8973 при нормальных условиях', ],
        ['М1 ГОСТ Р 53656 (доп)', f'{gost53656_permiability_alt.magnitude:.2e}',gost53656_permiability_alt.units, 'Соответствует значению по ГОСТ 8973', gost53656_permiability_alt.to(gost8973_UOM).magnitude],
        ['М2 ГОСТ 8973', gost8973_permiability_m2.magnitude, gost8973_permiability_m2.units, '', gost8973_permiability_m2.magnitude],
        ['М2 ГОСТ 23553', gost23553_permiability_m2.magnitude, gost23553_permiability_m2.units, '', (gost23553_permiability_m2 * thickness_PVC).to(gost8973_UOM).magnitude],
        ['М2 ГОСТ Р 53656', f'{gost53656_permiability_m2.magnitude:.2}', gost53656_permiability_m2.units, 'Соответствует значению по ГОСТ 8973 при нормальных условиях', ],
    ],
    columns=['НД','Значение','Единица измерения', 'Примечание','м² / Па / сек. (ГОСТ 8973)']
    )
pd.set_option('display.max_colwidth', 150)
display(dates)

,НД,Значение,Единица измерения,Примечание,м² / Па / сек. (ГОСТ 8973)
0,ИД: ПВХ 1,0.198,milliliter / day / meter / pascal,,2.291667e-12
1,ИД: ПВХ 2,3.1e-08,milliliter / day / meter / pascal,,3.587963e-19
2,ИД: Максимум по ГОСТ 23553,2e-12,meter / pascal / second,Перевод в единицы по ГОСТ 8973 при толщине 0.7 millimeter,1.400000e-15
3,М1 ГОСТ 8973,0.02,meter ** 2 / pascal / second,,2.000000e-02
4,М1 ГОСТ 23553,28.5714,meter / pascal / second,"Соответсвует значению по ГОСТ 8973 при толщине 0.7 millimeter. Значение намного !!! больше, чем допускает ГОСТ 23553 (2*10**(-12) м/(Па * сек.)). ...",2.000000e-02
5,М1 ГОСТ Р 53656,0.83,mole / meter / pascal / second,Соответствует значению по ГОСТ 8973 при нормальных условиях,NaN
6,М1 ГОСТ Р 53656 (доп),1.75e+17,centimeter ** 3 * millimeter / day / meter ** 2 / standard_atmosphere,Соответствует значению по ГОСТ 8973,2.000000e-02
7,М2 ГОСТ 8973,1.56199e-14,meter ** 2 / pascal / second,,1.561991e-14
8,М2 ГОСТ 23553,2.23142e-11,meter / pascal / second,,1.561991e-14
9,М2 ГОСТ Р 53656,6.5e-13,mole / meter / pascal / second,Соответствует значению по ГОСТ 8973 при нормальных условиях,NaN


In [26]:
dates.to_csv('results.csv')
dates.to_excel('results.xls')